In [1]:
### import os
import cv2
import numpy as np
import time,pdb
from torch.multiprocessing import Pool
from utils.nms_wrapper import nms
from utils.timer import Timer
from configs.CC import Config
import argparse
from layers.functions import Detect, PriorBox
from m2det import build_net
from data import BaseTransform
from utils.core import *
from utils.pycocotools.coco import COCO

print_info(' ----------------------------------------------------------------------\n'
           '|                       M2Det Demo Program                             |\n'
           ' ----------------------------------------------------------------------', ['yellow','bold'])

global cfg

im_path = "/home/jess/creDa_test_private_final/JPEGImages/All/"
cam = -1
show = False
checkpoint_id=43

cfg = Config.fromfile("configs/m2det512_mobilenetv3.py")
anchor_config = anchors(cfg)
print_info('The Anchor info: \n{}'.format(anchor_config))
priorbox = PriorBox(anchor_config)
net = build_net('test',
                size = cfg.model.input_size,
                config = cfg.model.m2det_config)
init_net(net, cfg, "weights/mbv3.pth")
fp = open('m2_mobilenetv3_%d.csv' % (checkpoint_id), 'w')

print_info('===> Finished constructing and loading model',['yellow','bold'])
net.eval()
with torch.no_grad():
    priors = priorbox.forward()
    if cfg.test_cfg.cuda:
        net = net.cuda()
        priors = priors.cuda()
        cudnn.benchmark = True
    else:
        net = net.cpu()
_preprocess = BaseTransform(cfg.model.input_size, cfg.model.rgb_means, (2, 0, 1))
detector = Detect(cfg.model.m2det_config.num_classes, cfg.loss.bkg_label, anchor_config)

def _to_color(indx, base):
    """ return (b, r, g) tuple"""
    base2 = base * base
    b = 2 - indx / base2
    r = 2 - (indx % base2) / base
    g = 2 - (indx % base2) % base
    return b * 127, r * 127, g * 127
base = int(np.ceil(pow(cfg.model.m2det_config.num_classes, 1. / 3)))
colors = [_to_color(x, base) for x in range(cfg.model.m2det_config.num_classes)]
cats = [_.strip().split(',')[-1] for _ in open('id.txt','r').readlines()]
labels = tuple(['__background__'] + cats)

def draw_detection(im, bboxes, scores, cls_inds, fps, thr=0.2):
    imgcv = np.copy(im)
    h, w, _ = imgcv.shape
    for i, box in enumerate(bboxes):
        if scores[i] < thr:
            continue
        cls_indx = int(cls_inds[i])
        box = [int(_) for _ in box]
        thick = int((h + w) / 300)
        cv2.rectangle(imgcv,
                      (box[0], box[1]), (box[2], box[3]),
                      colors[cls_indx], thick)
        mess = '%s: %.3f' % (labels[cls_indx], scores[i])
        cv2.putText(imgcv, mess, (box[0], box[1] - 7),
                    0, 1e-3 * h, colors[cls_indx], thick // 3)
        if fps >= 0:
            cv2.putText(imgcv, '%.2f' % fps + ' fps', (w - 160, h - 15), 0, 2e-3 * h, (255, 255, 255), thick // 2)

    return imgcv

im_fnames = sorted((fname for fname in os.listdir(im_path) if os.path.splitext(fname)[-1] == '.jpg'))
im_fnames = list(os.path.join(im_path, fname) for fname in im_fnames)
len1 = len(im_fnames)
cnt=0
fpss=0.0
fp.write('image_filename,label_id,x,y,w,h,confidence\n')
print('Found %d test images and we are going to predict them...' % (len1))
for z in range(len1):
    fname=im_fnames[z]
    image = cv2.imread(fname, cv2.IMREAD_COLOR)
    
    loop_start = time.time()
    w,h = image.shape[1],image.shape[0]
    img = _preprocess(image).unsqueeze(0)
    if cfg.test_cfg.cuda:
        img = img.cuda()
    scale = torch.Tensor([w,h,w,h])
    out = net(img)
    boxes, scores = detector.forward(out, priors)
    boxes = (boxes[0]*scale).cpu().numpy()
    scores = scores[0].cpu().numpy()
    allboxes = []
    for j in range(1, cfg.model.m2det_config.num_classes):
        inds = np.where(scores[:,j] > cfg.test_cfg.score_threshold)[0]
        if len(inds) == 0:
            continue
        c_bboxes = boxes[inds]
        c_scores = scores[inds, j]
        c_dets = np.hstack((c_bboxes, c_scores[:, np.newaxis])).astype(np.float32, copy=False)
        soft_nms = cfg.test_cfg.soft_nms
#         keep = nms(c_dets, cfg.test_cfg.iou, force_cpu = soft_nms)
#         keep = keep[:cfg.test_cfg.keep_per_class]
#         c_dets = c_dets[keep, :]
        allboxes.extend([_.tolist()+[j] for _ in c_dets])

    loop_time = time.time() - loop_start
    if allboxes==[]:
        continue
    allboxes = np.array(allboxes)
    boxes = allboxes[:,:4]
    scores = allboxes[:,4]
    cls_inds = allboxes[:,5]
    fp.write(''.join(['{},{},{},{:.3f}\n'.format( \
            fname.split('/')[-1],int(oo),'%d,%d,%d,%d' % (o[0],o[1],o[2]-o[0],o[3]-o[1]),ooo) for o,oo,ooo in zip(boxes,cls_inds,scores)]))
    fps = 1.0 / float(loop_time)
    fpss = fpss+fps
    im2show = draw_detection(image, boxes, scores, cls_inds, fps)
    cv2.imwrite('detected/%s' % (os.path.basename(fname)), im2show)
    if z % (len1//50)==0:
        print("Progress: %.4f" % (z/len1 * 100.0), end=", ")
print("###############################")
print("Averaged FPS:", fpss/len1)
print("###############################")
fp.close()
del net, image, allboxes, boxes, scores, detector
torch.cuda.empty_cache() 
print('Done')


 ----------------------------------------------------------------------
|                       M2Det Demo Program                             |
 ----------------------------------------------------------------------
The Anchor info: 
{'feature_maps': [64, 32, 16, 8, 4, 2], 'min_dim': 512, 'steps': [8, 16, 32, 64, 107, 320], 'min_sizes': [40.96, 76.8, 168.96, 261.12, 353.28, 445.44], 'max_sizes': [76.8, 168.96, 261.12, 353.28, 445.44, 537.6], 'aspect_ratios': [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]], 'variance': [0.1, 0.2], 'clip': True}
===> Constructing M2Det model
Loading resume network...
===> Finished constructing and loading model
Found 3000 test images and we are going to predict them...
Progress: 0.0000, Progress: 2.0000, Progress: 4.0000, Progress: 6.0000, Progress: 8.0000, Progress: 10.0000, Progress: 12.0000, Progress: 14.0000, Progress: 16.0000, Progress: 18.0000, Progress: 20.0000, Progress: 22.0000, Progress: 24.0000, Progress: 26.0000, Progress: 28.0000, Progress

In [ ]:
os._exit(00)
